In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

In [44]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e3/train.csv")

test_data = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv")


In [45]:
train_data.describe


<bound method NDFrame.describe of         id  day  pressure  maxtemp  temparature  mintemp  dewpoint  humidity  \
0        0    1    1017.4     21.2         20.6     19.9      19.4      87.0   
1        1    2    1019.5     16.2         16.9     15.8      15.4      95.0   
2        2    3    1024.1     19.4         16.1     14.6       9.3      75.0   
3        3    4    1013.4     18.1         17.8     16.9      16.8      95.0   
4        4    5    1021.8     21.3         18.4     15.2       9.6      52.0   
...    ...  ...       ...      ...          ...      ...       ...       ...   
2185  2185  361    1014.6     23.2         20.6     19.1      19.9      97.0   
2186  2186  362    1012.4     17.2         17.3     16.3      15.3      91.0   
2187  2187  363    1013.3     19.0         16.3     14.3      12.6      79.0   
2188  2188  364    1022.3     16.4         15.2     13.8      14.7      92.0   
2189  2189  365    1013.8     21.2         19.1     18.0      18.0      89.0   

     

In [46]:
features = list(train_data)
print(features[:-1])

['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']


In [47]:
X_train = train_data[features[1:-1]]
Y_train = train_data[features[-1]]
X_test = test_data[features[1:-1]]

display(X_train)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8
...,...,...,...,...,...,...,...,...,...,...,...
2185,361,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,22.1
2186,362,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,35.3
2187,363,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,32.9
2188,364,1022.3,16.4,15.2,13.8,14.7,92.0,93.0,0.1,40.0,18.0


In [48]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

print("Imputation complete using IterativeImputer!")

Imputation complete using IterativeImputer!


In [49]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)

print("Imputation complete using IterativeImputer!")

Imputation complete using IterativeImputer!


In [50]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [51]:
# Ensure your data is in NumPy array format
X_train = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train
Y_train = Y_train.to_numpy() if hasattr(Y_train, 'to_numpy') else Y_train
X_test = X_test.to_numpy() if hasattr(X_test, 'to_numpy') else X_test

In [52]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [53]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification


# ✅ Split into Train & Validation Sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# ✅ Define SVM Model
model = SVC(kernel="rbf", probability=True)  # ✅ RBF Kernel (Non-linear)
# Use kernel="linear" for Linear SVM

# ✅ Train SVM Model
model.fit(X_train, Y_train)

# ✅ Make Predictions (Probability Output)
Y_pred_prob = model.predict_proba(X_val)[:, 1]
print(len(Y_pred_prob))
print(len(Y_val))
display(Y_pred_prob)
display(Y_val)

# ✅ Find Optimal Threshold using Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(Y_val, Y_pred_prob)
print(precision)
print(len(precision))
best_threshold = thresholds[np.argmax(precision[:-1] - recall[:-1])]
print(f"Optimal Threshold Found: {best_threshold:.4f}")

# ✅ Convert Probabilities to Binary Class Labels (Using Best Threshold)
Y_pred = (Y_pred_prob > best_threshold).astype(int)

# ✅ Evaluate Model Performance
accuracy = accuracy_score(Y_val, Y_pred)
print(f"SVM Accuracy Score with Optimized Threshold: {accuracy:.4f}")


438
438


array([0.90058217, 0.89146386, 0.77829739, 0.87783029, 0.9127037 ,
       0.95205957, 0.91764601, 0.89219475, 0.86407776, 0.89653494,
       0.94375116, 0.93685008, 0.80990379, 0.08963305, 0.26804863,
       0.87445083, 0.92726559, 0.84499622, 0.86799366, 0.04439234,
       0.56233918, 0.9080198 , 0.10842484, 0.89093395, 0.17078371,
       0.92352787, 0.93261815, 0.82745699, 0.92171229, 0.90037893,
       0.93886839, 0.89439444, 0.91720176, 0.05729993, 0.07566557,
       0.92912886, 0.92828803, 0.89424324, 0.81438205, 0.31359623,
       0.91154026, 0.9098144 , 0.94314431, 0.93732547, 0.90261394,
       0.9053687 , 0.87015607, 0.90364416, 0.24668198, 0.89180243,
       0.93052412, 0.10898316, 0.91460619, 0.9100199 , 0.82775401,
       0.79044943, 0.89803844, 0.91677124, 0.92376479, 0.81199421,
       0.57335729, 0.93150314, 0.94007915, 0.92841118, 0.95061301,
       0.90685755, 0.89124946, 0.94133883, 0.91995124, 0.79630786,
       0.92768903, 0.94665696, 0.88556237, 0.9190809 , 0.20882

289     1
1692    1
1590    1
383     1
1818    1
       ..
2073    1
744     1
879     1
1978    0
1170    1
Name: rainfall, Length: 438, dtype: int64

[0.7283105  0.72997712 0.73165138 0.73333333 0.73502304 0.73672055
 0.73842593 0.74013921 0.73953488 0.74125874 0.74299065 0.74473068
 0.74647887 0.74588235 0.74764151 0.74940898 0.75118483 0.75296912
 0.7547619  0.75656325 0.75837321 0.76019185 0.76201923 0.76385542
 0.76570048 0.76755448 0.76941748 0.77128954 0.77317073 0.77506112
 0.77696078 0.77886978 0.78078818 0.78271605 0.78465347 0.7866005
 0.78855721 0.79052369 0.7925     0.79448622 0.79648241 0.79848866
 0.80050505 0.80253165 0.80203046 0.80152672 0.80357143 0.8056266
 0.80769231 0.80719794 0.80670103 0.80878553 0.80829016 0.81038961
 0.8125     0.81462141 0.81675393 0.81889764 0.82105263 0.82058047
 0.82275132 0.82493369 0.82446809 0.824      0.82352941 0.82573727
 0.82526882 0.82749326 0.82972973 0.83197832 0.83152174 0.83106267
 0.83333333 0.83561644 0.83791209 0.84022039 0.84254144 0.84487535
 0.84722222 0.84679666 0.84916201 0.8487395  0.84831461 0.85070423
 0.85310734 0.85552408 0.85795455 0.86039886 0.86285714 0.862464

In [54]:
dtest = xgb.DMatrix(X_test)

NameError: name 'xgb' is not defined

In [ ]:
ids = test_data["id"]

In [ ]:
display(df)

In [55]:
predictions = model.predict(dtest)
pred = (predictions > 0.8).astype(int)

df = pd.DataFrame(pred, columns=["Predictions"])

# Save DataFrame to an Excel file
file_path = "/kaggle/working/model_predictions.xlsx"
df.to_excel(file_path, index=False)

# Save DataFrame to an Excel file
file_path = "/kaggle/working/ids.xlsx"
ids.to_excel(file_path, index=False)

print(f"Excel file saved as {file_path}")

NameError: name 'dtest' is not defined